# LT-IDS Trainer

In [ ]:
import glob
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
import joblib

In [ ]:
FEATURES = [
    'Init_Win_bytes_forward',
    'Destination Port',
    'Packet Length Variance',
    'Average Packet Size',
    'Packet Length Std',
    'Max Packet Length',
    'Subflow Fwd Bytes',
    'Bwd Packet Length Max',
    'Fwd Packet Length Mean',
    'Bwd Packet Length Mean',
    'Fwd Packet Length Min',
    'Bwd Packet Length Std',
    'Bwd Packet Length Min',
    'Init_Win_bytes_backward',
    'Fwd Packet Length Std',
    'Packet Length Mean',
    'Fwd Header Length',
    'Fwd Packet Length Max',
    'Fwd Header Length.1',
    'Bwd Header Length'
]
EXTRA = ["Label"]

In [ ]:
files = glob.glob("/content/drive/MyDrive/CSV2017/*.csv")
df_list = []
for path in tqdm(files, desc="Loading CSVs"):
    df_chunk = pd.read_csv(
        path,
        usecols=FEATURES + EXTRA,
        skipinitialspace=True,
        na_values=["NaN", "Infinity"],
        low_memory=False
    )
    df_list.append(df_chunk)
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df = df[df["Label"] != "Label"]
df[FEATURES] = df[FEATURES].apply(pd.to_numeric, errors="coerce")
df["y"] = (df["Label"] != "BENIGN").astype(int)

In [ ]:
X = df[FEATURES]
y = df["y"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))

In [ ]:
param_grid = {
    "max_depth":        [5, 10, 15, None],
    "min_samples_leaf": [1, 5, 10],
    "class_weight":     [{0:1,1:w} for w in [1, 2, 5]]
}

halving_cv = HalvingGridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    factor=3,
    resource="n_samples",
    scoring="f1",
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1
)
halving_cv.fit(X_train, y_train)

print("Best params:", halving_cv.best_params_)
best_dt = halving_cv.best_estimator_
print(classification_report(y_test, best_dt.predict(X_test), digits=4))

In [ ]:
joblib.dump(best_dt, "dtthree_ids_trained.joblib")